In [1]:
from nmon import *

 /Users/nikolaygusarov/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py: 34

In [2]:
import numpy as np

def transmon_hamiltonian_phase_basis(EJ, EC, Phi, n_g, M):
    """
    Generate the Hamiltonian matrix for a transmon qubit in the phase basis using spectral methods.

    Parameters:
    EJ (float): Josephson energy.
    EC (float): Charging energy.
    Phi (float): External flux in units of the flux quantum (Φ/Φ0).
    n_g (float): Offset charge.
    M (int): Number of discretization points in the phase basis.

    Returns:
    np.ndarray: The Hamiltonian matrix.
    """
    # Discretized phase basis points
    theta = np.linspace(0, 2 * np.pi, M, endpoint=False)

    # Charge number n ranging from -M//2 to M//2
    n = np.arange(-M//2, M//2)
    if M % 2 == 0:
        n = np.arange(-M//2, M//2)
    else:
        n = np.arange(-(M-1)//2, (M+1)//2)

    # Kinetic energy operator in the charge basis
    N = 4 * EC * (n - n_g)**2

    # Potential energy term in the phase basis
    V = -EJ * np.cos(theta - 2 * np.pi * Phi)

    # Fourier matrices
    F = np.exp(1j * np.outer(n, theta)) / np.sqrt(M)
    F_dag = np.conj(F).T

    # Transform kinetic energy to phase basis
    H_C = F_dag @ np.diag(N) @ F

    # Potential energy operator (diagonal in phase basis)
    H_J = np.diag(V)

    # Total Hamiltonian
    H = H_C + H_J

    # Ensure the Hamiltonian is Hermitian
    H = (H + H.conj().T) / 2

    return H

# Example usage
EJ = 1.5   # Josephson energy
EC = 0.5   # Charging energy
Phi = 0.0  # External flux (units of Φ/Φ0)
n_g = 0.0  # Offset charge
M = 101    # Number of discretization points in the phase basis

hamiltonian_matrix_phase = transmon_hamiltonian_phase_basis(EJ, EC, Phi, n_g, M)

# Compute eigenvalues
eigenvalues_phase = np.linalg.eigvalsh(hamiltonian_matrix_phase)

# Display the first few eigenvalues
print("Eigenvalues from phase basis:", eigenvalues_phase[:10])


Eigenvalues from phase basis: [-0.46840925  1.90714544  2.37338973  8.03659174  8.03875592 18.01608185
 18.01608262 32.0089303  32.0089303  50.00568225]


In [3]:
def transmon_hamiltonian_charge_basis(EJ, EC, n_cut, Phi, n_g):
    """
    Generate the Hamiltonian matrix for a transmon qubit in the charge basis.

    Parameters:
    EJ (float): Josephson energy.
    EC (float): Charging energy.
    n_cut (int): Cut-off for the charge basis.
    Phi (float): External flux in units of the flux quantum (Φ/Φ0).
    n_g (float): Offset charge.

    Returns:
    np.ndarray: The Hamiltonian matrix.
    """
    # Define the charge states
    n = np.arange(-n_cut, n_cut + 1)
    dim = len(n)
    H = np.zeros((dim, dim), dtype=complex)

    # Charging energy term
    H += np.diag(4 * EC * (n - n_g)**2)

    # Josephson energy term
    for i in range(dim):
        if i + 1 < dim:
            H[i, i + 1] += -EJ / 2 * np.exp(-2j * np.pi * Phi)
            H[i + 1, i] += -EJ / 2 * np.exp(2j * np.pi * Phi)
    return H.real

# Example usage
n_cut = (M - 1) // 2  # Ensure that dimension matches M
hamiltonian_matrix_charge = transmon_hamiltonian_charge_basis(EJ, EC, n_cut, Phi, n_g)

# Compute eigenvalues
eigenvalues_charge = np.linalg.eigvalsh(hamiltonian_matrix_charge)

# Compare eigenvalues
print("Eigenvalues from charge basis:", eigenvalues_charge[:10])
print("Eigenvalues from phase basis:", eigenvalues_phase[:10])


Eigenvalues from charge basis: [-0.46840925  1.90714544  2.37338973  8.03659174  8.03875592 18.01608185
 18.01608262 32.0089303  32.0089303  50.00568225]
Eigenvalues from phase basis: [-0.46840925  1.90714544  2.37338973  8.03659174  8.03875592 18.01608185
 18.01608262 32.0089303  32.0089303  50.00568225]
